In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai_api_key=os.getenv("OPENAI_API_KEY")
openai = OpenAI()

OPENAI_MODEL = "gpt-4.1-mini"

In [ ]:
system_prompt = """
    You are a helpful assistant. Your specialisation is to help customers book tickets. 
    You have a good knack of finding great offers to land customers in good deals.
    You work for a company called Flight AI
"""

In [ ]:
ticket_prices = {
    "bangalore": "5949",
    "mumbai": "4500",
    "delhi": "6200",
    "chennai": "3800",
    "kolkata": "7100",
    "hyderabad": "4100",
    "pune": "4700",
    "goa": "5500",
    "jaipur": "5800",
    "ahmedabad": "4900",
    "kochi": "3200",
    "trivandrum": "3400",
    "varanasi": "6700",
    "lucknow": "6100",
    "chandigarh": "6300",
    "mysore": "5100",
    "indore": "4300",
    "surat": "4600",
    "bhopal": "4400",
    "patna": "6900",
    "visakhapatnam": "5200"
}

In [ ]:
def get_ticket_prices(destination_city):
    print(f"get_ticket_prices tool called. destination city -> {destination_city}")
    if destination_city is None or destination_city.strip() == "":
        raise ValueError("Destination city cannot be None or blank")
    
    price = ticket_prices.get(destination_city.lower(), f"sorry, we do not operator to {destination_city}. ticket price not found")

    return f"The price of a ticket to {destination_city} is {price}"

In [ ]:
tools = [{"type": "function", "function": ticket_price_tool}]


In [ ]:
def chat_window_ui(user_message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_message}]

    response = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        tools=tools
    )
    
    if response.choices[0].finish_reason == 'tool_calls':
        tools_message = response.choices[0].message
        tools_response = handle_tools_call(tools_message)
        messages.append(tools_message)
        messages.append(tools_response)

        for message in messages:
            print(message)
            
        response = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages)
        
    return response.choices[0].message.content

In [ ]:
def handle_tools_call(tools_message):
    
    tool_call = tools_message.tool_calls[0]
    tools_function = tool_call.function
    tools_call_id = tool_call.id
    
    if tools_function.name == 'get_ticket_prices':
        function_arguments = json.loads(tools_function.arguments)
        destination_city = function_arguments.get("destination_city", "no city found")
        ticket_price_or_error = get_ticket_prices(destination_city=destination_city)
        response = {
            "role": "tool",
            "content": ticket_price_or_error,
            "tool_call_id": tools_call_id
        }

        return response

In [132]:
gr.ChatInterface(fn=chat_window_ui).launch()

* Running on local URL:  http://127.0.0.1:7933
* To create a public link, set `share=True` in `launch()`.


messages before tools call -> [{'role': 'system', 'content': '\n    You are a helpful assistant. Your specialisation is to help customers book tickets. \n    You have a good knack of finding great offers to land customers in good deals.\n    You work for a company called Flight AI\n'}, {'role': 'user', 'content': 'book a ticket to trivandrum'}]
abc => Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_elZL7o0V9FjCfKUa3ntlSGn4', function=Function(arguments='{"destination_city":"trivandrum"}', name='get_ticket_prices'), type='function')]))
get_ticket_prices tool called. destination city -> trivandrum
{'role': 'system', 'content': '\n    You are a helpful assistant. Your specialisation is to help customers book tickets. \n    You have a good knack of finding great offers to land customers in good deal

Traceback (most recent call last):
  File "/Users/rijuvijayan/1workspace/learning/udemy/LLM_Engineering/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rijuvijayan/1workspace/learning/udemy/LLM_Engineering/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rijuvijayan/1workspace/learning/udemy/LLM_Engineering/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 2125, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rijuvijayan/1workspace/learning/udemy/LLM_Engineering/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1605, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^